<a href="https://colab.research.google.com/github/Dushmanthr/Sinhala_AI_Generated_Answer_Detection/blob/main/My_research_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sinling


In [ ]:
!pip install advertools

In [ ]:
!pip install translate

In [ ]:
!pip install pyenchant

In [ ]:
from sinling import SinhalaTokenizer as tokenizer,SinhalaStemmer as stemmer, POSTagger,preprocess, word_joiner,word_splitter
from nltk.tokenize import sent_tokenize,word_tokenize,TweetTokenizer
from nltk.probability import FreqDist
import advertools as adv
from pathlib import Path
import string

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn import linear_model

import codecs
import nltk
nltk.download('punkt')
import re

from nltk.corpus import stopwords
from collections import Counter

from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# %% read csv file data
import matplotlib.pyplot as plt
import seaborn as sns

file = pd.read_excel('Processed.xlsx')
#pd.read_csv(Path().joinpath('data','traindata2.csv'))
print(file['Answers'].head(11))
plt.rcParams['font.sans-serif']= "cmr10"
sns.countplot(x='Label',hue='Label',data=file)
plt.show()

In [ ]:
df = pd.read_excel('Processed.xlsx')

print(df['Answers'])

In [ ]:
stopwords_set = ["සහ","සමග","සමඟ","අහා","ආහ්","ආ","ඕහෝ","අනේ","අඳෝ","අපොයි","පෝ","අයියෝ","ආයි","ඌයි","චී","චිහ්","චික්","හෝ‍","දෝ",
                 "දෝහෝ","මෙන්","සේ","වැනි","බඳු","වන්","අයුරු","අයුරින්","ලෙස","වැඩි","ශ්‍රී","හා","ය","නිසා","නිසාවෙන්","බවට","බව","බවෙන්","නම්","වැඩි","සිට",
                 "දී","මහා","මහ","පමණ","පමණින්","පමන","වන","විට","විටින්","මේ","මෙලෙස","මෙයින්","ඇති","ලෙස","සිදු","වශයෙන්","යන","සඳහා","මගින්","හෝ‍",
                 "ඉතා","ඒ","එම","ද","අතර","විසින්","සමග","පිළිබඳව","පිළිබඳ","තුළ","බව","වැනි","මහ","මෙම","මෙහි","මේ","වෙත","වෙතින්","වෙතට","වෙනුවෙන්",
                 "වෙනුවට","වෙන","ගැන","නෑ","අනුව","නව","පිළිබඳ","විශේෂ","දැනට","එහෙන්","මෙහෙන්","එහේ","මෙහේ","ම","තවත්","තව","සහ","දක්වා","ට","ගේ",
                 "එ","ක","ක්","බවත්","බවද","මත","ඇතුලු","ඇතුළු","මෙසේ","වඩා","වඩාත්ම","නිති","නිතිත්","නිතොර","නිතර","ඉක්බිති","දැන්","යලි","පුන","ඉතින්",
                 "සිට","සිටන්","පටන්","තෙක්","දක්වා","සා","තාක්","තුවක්","පවා","ද","හෝ‍","වත්","විනා","හැර","මිස","මුත්","කිම","කිම්","ඇයි","මන්ද","හෙවත්",
                 "නොහොත්","පතා","පාසා","ගානෙ","තව","ඉතා","බොහෝ","වහා","සෙද","සැනින්","හනික","එම්බා","එම්බල","බොල","නම්","වනාහි","කලී","ඉඳුරා",
                 "අන්න","ඔන්න","මෙන්න","උදෙසා","පිණිස","සඳහා","රබයා","නිසා","එනිසා","එබැවින්","බැවින්","හෙයින්","සේක්","සේක","ගැන","අනුව","පරිදි","විට",
                 "තෙක්","මෙතෙක්","මේතාක්","තුරු","තුරා","තුරාවට","තුලින්","නමුත්","එනමුත්","වස්",'මෙන්',"ලෙස","පරිදි","එහෙත්"]


In [ ]:
print(stopwords_set)

In [ ]:
translate_words_dict = {
    "unp": "එක්සත් ජාතික පක්ෂය",
    "muslim": "මුස්ලිම්",
    "srilankanpolitics": "ශ්‍රී ලංකන් දේශපාලනය",
    "council": "සභාව",
    "sinhala": "සිංහල",
    "buddhist": "බෞද්ධ",
    "buddhism": "බුද්ධාගම",
    "srilanka": "ශ්‍රී ලංකාව",
    "racist": "ජාතිවාදී",
    "presidentialfirst": "පළමු ජනාධිපති",
    "feeling": "හැඟීම",
    "feminist": "ස්ත්‍රීවාදී",
    "loved": "ආදරය කළා",
    "team": "කණ්ඩායම",
    "tclsl":"ට්විටර් ක්‍රිකට් ලීගය ශ්‍රී ලංකාව",
    "pongal": "පොංගල්",
    "pongalfestival": "පොංගල් උත්සවය",
    "women": "කාන්තා",
    "nextpresidentinsl": "ශ්‍රී ලංකාවේ මීළඟ ජනාධිපති ",
    "seventhexecutivepresident": "හත්වන විධායක සභාපති",
    "hate": "වෛරය",
    "love": "ආදරය",
    "angry": "තරහයි",
    "doctor": "ඩොක්ටර්",
    "ltte": "එල්ටීටීඊය",
    "lka": "‍ශ්‍රී ලංකාව",
    "hurt": "රිදෙනවා",
    "typo": "යතුරු ලියනය",
    "racial": "වාර්ගික",
    "hatred": "වෛරය",
    "halal": "හලාල්",
    "wicket": "කඩුල්ල",
    "taker": "ටේකර්",
    "indoor": "ගෘහස්ථ",
    "attacker": "ප්‍රහාරකයා",
    "attack": "ප්රහාරය",
    "spikers": "ස්පිකර්ස්",
    "training": "පුහුණුව",
    "final": "අවසාන",
    "match": "තරගය",
    "tournament": "තරඟාවලිය",
    "youth": "තරුණ",
    "amen": "ආමෙන්",
    "enough": "ඇති",
    "standagainstracism": "ජාතිවාදයට එරෙහිව නැගී සිටින්න"
}

def translate_to_sinhala(word: str) -> str:
  word = word.lower()
  if word in translate_words_dict:
        return translate_words_dict[word]
  return word

In [ ]:
from sinling import SinhalaTokenizer as tokenizer,SinhalaStemmer as stemmer, POSTagger,preprocess, word_joiner,word_splitter
stemmer = stemmer()

def stem_word(word: str) -> str:
    word= translate_to_sinhala(word)
    """
    Stemming words
    :param word: word
    :return: stemmed word
    """
    if len(word) < 4:
        return word

    # remove 'ට'
    if word[-1] == 'ට':
        return word[:-1]

    # remove 'ම'
    if word[-1] == 'ම':
        return word[:-1]

    # remove 'ද'
    if word[-1] == 'ද':
        return word[:-1]

    # remove 'ටත්'
    if word[-3:] == 'ටත්':
        return word[:-3]

    # remove 'එක්'
    if word[-3:] == 'ෙක්':
        return word[:-3]

    # remove 'යේ'
    if word[-2:] == 'යේ':
        return word[:-2]

    # remove 'ගෙ' (instead of ගේ because this step comes after simplifying text)
    if word[-2:] == 'ගෙ':
        return word[:-2]

    # remove 'එ'
    if word[-1:] == 'ෙ':
        return word[:-1]

    # remove 'ක්'
    if word[-2:] == 'ක්':
        return word[:-2]

    # remove 'වත්'
    if word[-3:] == 'වත්':
        return word[:-3]

    word=stemmer.stem(word)
    word=word[0]


    # else
    return word

In [ ]:


def filter_stop_words(sentences):
    filtered_sentences = []
    detokenizer = Detok()
    for sentence in sentences:
        tokenized_sentence = word_tokenize(sentence)
        filtered_sentence = [word for word in tokenized_sentence if word not in stopwords_set]
        filtered_sentence = []
        for w in tokenized_sentence:
            if w not in stopwords_set:
                filtered_sentence.append(stem_word(w))
        filtered_sentences.append(filtered_sentence)
    return filtered_sentences





In [ ]:
def detokenize(filtered_sentences):
    detokenized_sentences = []
    for sentence in filtered_sentences:
        detokenized_sentences.append(TreebankWordDetokenizer().detokenize(sentence))
    return detokenized_sentences

    print(detokenized_sentences)


In [ ]:
df.info()

In [ ]:
df['Answers'] = df['Answers'].astype(str)

In [ ]:
nltk.download('punkt_tab')
filtered_sentences = filter_stop_words(df['Answers'])
detokenized_sentences = detokenize(filtered_sentences)

In [ ]:
detokenized_sentences

In [ ]:
simplify_characters_dict = {
    # Consonant
    "ඛ": "ක",
    "ඝ": "ග",
    "ඟ": "ග",
    "ඡ": "ච",
    "ඣ": "ජ",
    "ඦ": "ජ",
    "ඤ": "ඥ",
    "ඨ": "ට",
    "ඪ": "ඩ",
    "ණ": "න",
    "ඳ": "ද",
    "ඵ": "ප",
    "භ": "බ",
    "ඹ": "බ",
    "ශ": "ෂ",
    "ළ": "ල",

    # Vowels
    "ආ": "අ",
    "ඈ": "ඇ",
    "ඊ": "ඉ",
    "ඌ": "උ",
    "ඒ": "එ",
    "ඕ": "ඔ",

}

In [ ]:
def get_simplified_character(character: str) -> str:
    if len(character) != 1:
        raise TypeError("character should be a string with length 1")
    try:
        return simplify_characters_dict[character]
    except KeyError:
        return character

In [ ]:
def simplify_sinhalese_text(text: str) -> str:
    """
    simplify
    :param text:
    :return:
    """
    modified_text = ""
    for c in text:
        modified_text += get_simplified_character(c)
    return modified_text

In [ ]:
df['filtered_sentence'] = detokenized_sentences
df['filtered_sentence'] = df['filtered_sentence'].apply(simplify_sinhalese_text).tolist()
#df['filtered_sentence'] = df['Text'].apply(remove_english_words).tolist()

#df.to_csv('SubjectivityTagged.csv')
df.to_excel('stem.xlsx')

# Data Cleaning

In [ ]:
df = pd.read_excel('stem.xlsx')


In [ ]:
df['Text_cleaned'] = df['Answers'].str.replace('**',' ')
df['Text_cleaned'] = df['Text_cleaned'].str.replace('?',' ')

df['Text_cleaned'][1188]

In [ ]:
# english-sinhala dictionary
dictionary = {}
df_translate= pd.read_csv('en-sinhala dictionary.csv')
dictionary_file = df_translate["En,sinhala"]

for line in dictionary_file:
    key, value = line.strip().split(",")
    dictionary[key] = value

In [ ]:
# to check whether the string contains English words(any)

def translate_english(x):
    for word in x.split():
        word_cleaned = "".join(char for char in word if char not in string.punctuation)  # Remove punctuation
        if re.match('[a-zA-Z]', word_cleaned):  # Check if the word contains English letters
            word_lower = word_cleaned.lower()  # Convert to lowercase for dictionary lookup
            translated_word = dictionary.get(word_lower, '')  # Translate using the dictionary
            x = x.replace(word, translated_word)
    return x

In [ ]:
df.info()

In [ ]:

def clean_data(dataframe):
    # Drop duplicate rows
    dataframe.drop_duplicates(subset='Answers', inplace=True)

    #punctuation removal
    string_text = dataframe['Text_cleaned'].str
    dataframe['Text_cleaned'] = string_text.translate(str.maketrans('', '', string.punctuation))
    print(string_text)

    # coerced entire coloumn to str dtype
    dataframe['Text_cleaned'] = dataframe['Text_cleaned'].astype(str)

    #translate English to sinhala
    df['Text_cleaned'] = df['Text_cleaned'].apply(translate_to_sinhala).tolist()
    df['Text_cleaned'] = df['Text_cleaned'].apply(translate_english).tolist()

    # simplify sinhala characters
    df['Text_cleaned'] = df['Text_cleaned'].apply(simplify_sinhalese_text).tolist()

    # pos tagging
    #df['Text'] = df['Text'].apply(tagger.predict).tolist()

    #print("New shape:", dataframe.shape)
    return dataframe.head()

clean_data(df)
df

In [ ]:
df.to_excel('clean.xlsx')

In [ ]:
df = df.dropna(how='all')

In [ ]:

df.to_excel('clean.xlsx')

In [ ]:
# Convert 'Labels' column to 0 and 1
df['Label'] = df['Label'].map({'Human': 0, 'AI': 1})

In [ ]:

# Verify the conversion
print(df['Label'].value_counts())
df

In [ ]:
import nltk
nltk.download('punkt')  # Download tokenizer resources
from nltk.tokenize import word_tokenize

# Tokenize the 'Text_cleaned' column and create a new 'Tokenized' column
df['Tokenized'] = df['Text_cleaned'].apply(word_tokenize)

In [ ]:
print(df[['Text_cleaned', 'Tokenized']].head())

In [ ]:
df.to_excel('tokenized_data.xlsx')

#Feature **Extraction** | TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK Sinhala support (if necessary)
nltk.download('punkt')

# Step 1: Tokenize Sinhala text
df['Tokenized'] = df['Text_cleaned'].apply(word_tokenize)

# Step 2: Convert tokens back to sentences for TF-IDF
df['Text_reconstructed'] = df['Tokenized'].apply(lambda tokens: " ".join(tokens))

# Step 3: Initialize TF-IDF Vectorizer (customize stop words for Sinhala if needed)
tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)

# Step 4: Fit and transform the Sinhala text data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Text_reconstructed'])

# Step 5: Convert TF-IDF matrix to a DataFrame
tfidf_features = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

# Step 6: Combine TF-IDF features with the original dataset
df_with_tfidf = pd.concat([df, tfidf_features], axis=1)

# Save the TF-IDF-enhanced dataset
#output_file = "/mnt/data/tfidf_features_sinhala.xlsx"
#df_with_tfidf.to_excel(output_file, index=False)

#output_file


In [ ]:
df_with_tfidf.head()

In [ ]:
# Save the TF-IDF-enhanced dataset
output_file = "tfidf_features_sinhala.xlsx"
df_with_tfidf.to_excel(output_file)

In [ ]:
from sklearn.model_selection import train_test_split

X = tfidf_matrix  # Features from TF-IDF
y = df['Label']   # Labels: Human (0) or AI (1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Ensure inputs are in array format
X_train = np.array(X_train.toarray())  # Convert sparse matrix to dense array
X_test = np.array(X_test.toarray())

# Define the ANN model
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),  # Input layer with 128 neurons
    Dropout(0.2),  # Dropout for regularization
    Dense(64, activation='relu'),  # Hidden layer with 64 neurons
    Dropout(0.2),  # Dropout for regularization
    Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Test the model
y_pred_prob = model.predict(X_test)  # Predicted probabilities
y_pred = (y_pred_prob > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.5657 - loss: 0.6864 - val_accuracy: 0.8119 - val_loss: 0.6408
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9235 - loss: 0.5174 - val_accuracy: 0.8339 - val_loss: 0.4222
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9799 - loss: 0.1241 - val_accuracy: 0.8245 - val_loss: 0.3911
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9946 - loss: 0.0303 - val_accuracy: 0.8150 - val_loss: 0.4164
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9963 - loss: 0.0156 - val_accuracy: 0.8182 - val_loss: 0.4410
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9998 - loss: 0.0056 - val_accuracy: 0.8088 - val_loss: 0.4605
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 0.8119 - val_loss: 0.4735
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 1.0000 - loss: 0.0028 - val_accuracy: 0.8056 - v